In [3]:
import requests
import lxml.html as lh
import unicodecsv
from lxml.cssselect import CSSSelector
import requests_cache
import json

requests_cache.install_cache('demo_cache')


In [123]:
s = requests.Session()
s.mount('http://', requests.adapters.HTTPAdapter(max_retries=10))

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))
    
def stringify_children(node):
    from lxml.etree import tostring
    from itertools import chain
    parts = ([node.text] +
            list(chain(*([c.text,
                          #tostring(c,method="text",encoding="unicode"),
                          c.tail] for c in node.getchildren()))) +
            [node.tail])
    # filter removes possible Nones in texts and tails
    return ''.join(filter(None, parts))
    
def extract_data(x):
    return stringify_children(x).strip()

def extract_line(tr):
    return [x
            for x in 
            tr.xpath('./*[name()="th" or name()="td"]')]


def extract_table(t):
    trs = t.xpath('.//tr')
    headers = [extract_data(x) for x in extract_line(trs[0])]
    ds = []
    for tr in trs[1:]:
         ds.append(dict(zip(headers,extract_line(tr))))
    return ds
    

In [124]:
root_url = 'https://data.fis-ski.com/global-links/all-fis-results.html?place_search=&seasoncode_search=all&sector_search=AL&date_search=&gender_search=&category_search=WSC&codex_search=&nation_search=&disciplinecode_search=&date_from=01&search=Search&limit=100'

root1 = lh.fromstring(s.get(root_url).content)
table = root1.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')[0]

events = [x for x in extract_table(table) if len(x.keys())>1]
for event in events:
    keys = event.keys()
    event['link'] = event['Event'].xpath('.//a/@href')[0]
    for k in keys:
        event[k] = extract_data(event[k])
    print event['link']
    root2 = lh.fromstring(s.get(event['link']).content)
    table = root2.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')[0]
    subevents = [x for x in extract_table(table) if 'Discipline' in x.keys()]
    for se in subevents:
        keys = se.keys()
        try:
            se['url'] = se['Discipline'].xpath('.//a/@href')[0]
        except IndexError:
            print 'not found', se.keys(), lh.tostring(se['Discipline']), event['link']
            pass
            #print 'could not find link'
        for k in keys:
            se[k] = extract_data(se[k])
        for k in event.keys():
            se['event_%s' % k] = event[k]
    event['subevents'] = subevents
    
all_subevents = [x for x in event['subevents'] for event in events if 'url' in x]

https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL
not found ['', 'Discipline', 'Codex', 'Gender', 'Category', 'Nation', 'Download', 'Place', 'Comments', 'Date'] <td class="i0">Downhill training</td>
https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL
not found ['', 'Discipline', 'Codex', 'Gender', 'Category', 'Nation', 'Download', 'Place', 'Comments', 'Date'] <td class="i1">Downhill</td>
https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL
not found ['', 'Discipline', 'Codex', 'Gender', 'Category', 'Nation', 'Download', 'Place', 'Comments', 'Date'] <td class="i0">Downhill training</td>
https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL
not found ['', 'Discipline', 'Codex', 'Gender', 'Category', 'Nation', 'Download', 'Place', 'Comments', 'Date'] <td class="i1">Downhill</td>
https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=

In [111]:
#[x['link'] for x in events]
[(x['url'],x['event_link'],) for x in all_subevents]

[('https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030',
  'https://data.fis-ski.com/dynamic/event-details.html?event_id=7211&cal_suchsector=AL'),
 ('https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030',
  'https://data.fis-ski.com/dynamic/event-details.html?event_id=7211&cal_suchsector=AL'),
 ('https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030',
  'https://data.fis-ski.com/dynamic/event-details.html?event_id=7211&cal_suchsector=AL'),
 ('https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030',
  'https://data.fis-ski.com/dynamic/event-details.html?event_id=7211&cal_suchsector=AL'),
 ('https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030',
  'https://data.fis-ski.com/dynamic/event-details.html?event_id=7211&cal_suchsector=AL'),
 ('https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030',
  'https://data.fis-ski.com/dynamic/event-details.html?event_id=7211&cal_suchsector=AL'),
 ('https://data.

In [126]:

done_urls = []
all_data = []


if True: #with open('/media/sf_ddj/geschichten/2016-10-26-zeit-SpA/skilaeufer/refined/subpages.csv') as f:
    l = all_subevents
    for i,x in enumerate(l):
        if i%100==0:
            print i
        if x['url'] in done_urls:
            continue
        rq = s.get(x['url'])
        if rq.status_code!=200:
            print x['url']
            print rq
            continue
        print x['url']
        root = lh.fromstring(rq.content)
        tables = root.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')
        if len(tables)!=1:
            print x['url'], tables
            continue
        
        tbl = tables[0]
        for d in extract_table(tbl):
            for k in d:
                #print k
                d[k] = extract_data(d[k])
                
             #d = dict(zip(headers,extract_data(tr)))
            d['source_url']=x['url']
            for k in x.keys():
                d['subevent_%s' % k] = x[k]
            all_data.append(d)
        done_urls.append(x['url'])
        
        if i%100==0:
            with open('done_urls_2.json','w') as f2:
                json.dump(done_urls, f2)

            with open('all_data_2.json','w') as f2:
                json.dump(all_data, f2)
                
with open('done_urls.json','w') as f2:
    json.dump(done_urls, f2)

with open('all_data.json','w') as f2:
    json.dump(all_data, f2)

0
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14030
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14028
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14031
100
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14029


In [133]:
all_data[15]

{'Bib': u'',
 'Diff.': u'&nbsp',
 'FIS Code': u'-11175',
 'Name': u'WERSIN-LANTSCHNER Inge',
 'Nation': u'AUT',
 'Rank': u'2',
 'Run 1': u'',
 'Run 2': u'',
 'Total Time': u'',
 'Year': u'1905',
 'source_url': 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=14031',
 'subevent_': '',
 'subevent_Category': 'WSC',
 'subevent_Codex': '6003',
 'subevent_Comments': '',
 'subevent_Date': u'',
 'subevent_Discipline': 'Slalom',
 'subevent_Download': '',
 'subevent_Gender': '',
 'subevent_Nation': '',
 'subevent_Place': '',
 'subevent_event_CategoryCat.': 'WSC',
 'subevent_event_Codex': '',
 'subevent_event_Comments': '',
 'subevent_event_Date': '1.-28.2.1931',
 'subevent_event_Discipline': 'DH - SL',
 'subevent_event_Download': '',
 'subevent_event_Event': '',
 'subevent_event_GenderG.': 'L',
 'subevent_event_NationNat.': 'SUI',
 'subevent_event_Place': 'Muerren',
 'subevent_event_SectorSect.': 'AL',
 'subevent_event_Status': '',
 'subevent_event_link': 'https://data.fis-ski.com

In [77]:
all_data[:5]

[{'Bib': u'',
  'Diff.': u'&nbsp',
  'FIS Code': u'55198',
  'FIS Points': u'-4.00',
  'Name': u'DORFMEISTER Michaela',
  'Nation': u'AUT',
  'Rank': u'1',
  'Run 1': u'1:07.95',
  'Run 2': u'1:11.93',
  'Total Time': u'2:19.88',
  'Year': u'1973',
  'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'},
 {'Bib': u'',
  'Diff.': u'+0.83',
  'FIS Code': u'505483',
  'FIS Points': u'0.92',
  'Name': u'PAERSON Anja',
  'Nation': u'SWE',
  'Rank': u'2',
  'Run 1': u'1:08.09',
  'Run 2': u'1:12.62',
  'Total Time': u'2:20.71',
  'Year': u'1981',
  'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'},
 {'Bib': u'',
  'Diff.': u'+0.87',
  'FIS Code': u'515268',
  'FIS Points': u'1.16',
  'Name': u'NEF Sonja',
  'Nation': u'SUI',
  'Rank': u'3',
  'Run 1': u'1:09.04',
  'Run 2': u'1:11.71',
  'Total Time': u'2:20.75',
  'Year': u'1972',
  'source_url': u'http://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=13100'},
 {'Bi

In [8]:
print len(all_data), len(done_urls)

132313 3202


In [137]:
all_keys = set()
for x in all_data:
    for k in x.keys():
        all_keys.add(k)

In [140]:
all_keys = sorted(list(all_keys))

In [141]:
with open('fin_output_2.csv','w') as of:
    ow = unicodecsv.writer(of)
    ow.writerow(all_keys)
    for x in all_data:
        ow.writerow([x.get(k,'?????') for k in all_keys])